In [1]:
class LogNormal(object):

    def __init__(self, X=None, Mu=None, Sigma=None):
        self.X = X
        self.Mu = Mu
        self.Sigma = Sigma
        
    def pdf(self, X, Mu, Sigma):
        return (np.exp(-(np.log(X) - Mu)**2 / (2 * Sigma**2)) / (X * Sigma * np.sqrt(2 * np.pi)))

    def cdf(self, X, Mu, Sigma):
        from scipy.special import erf
        return 0.5+0.5*(erf((np.log(X)-Mu)/(np.sqrt(2)*Sigma)))
    
    def mean(self, Mu, Sigma):
        return np.exp(Mu+(0.5*(Sigma**2)))
    
    def median(self, Mu, Sigma=None):
        return np.exp(Mu)
    
    def mode(self, Mu, Sigma):
        return np.exp(Mu-(Sigma**2))
    
    def var(self, Mu, Sigma):
        return (np.exp(Sigma**2)-1)*np.exp(2*Mu+(Sigma**2))
    
    def std(self, Mu, Sigma):
        return np.sqrt((np.exp(Sigma**2)-1) * np.exp(2*Mu+(Sigma**2)))
    

    def quantile(self, Probs, Mu=None, Sigma=None): # all None's have to be last
        from scipy.special import erfinv
        
        probs = [Probs]
        probs = np.asanyarray(probs)
        if not Mu:
            Mu = 0
        if not Sigma:
            Sigma = 1.0

        quant = []
        for i,prob in enumerate(probs):
            stand_norm_quantile = (0 + 1.0 * np.sqrt(2) * erfinv(2* prob-1)) # (Mu + Sigma * np.sqrt(2) * erfinv(2* Prob-1)) 
            quant.append(2.71828**(Mu+(stand_norm_quantile*Sigma)))
        return quant[0]

Lognorm = LogNormal()

In [2]:
from __future__ import division
import sys, os
import numpy as np
import pandas as pd
from collections import OrderedDict
import pyfits

import matplotlib.pyplot as plt
%matplotlib inline

# import matplotlib.mlab as mlab
# from matplotlib.backends.backend_pdf import PdfPages
# from matplotlib.patches import Ellipse

from scipy.stats import norm, lognorm

/Users/KimiZ/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [7]:
modname      = 'grbm+blackb'
version      = '-01-'
detector     = 'GBMwLAT'

burst        = 'bn080916009'
data_dir     = "/Users/KimiZ/GRBs2/analysis/LAT/%s/integrated/"%burst
mod_dir      = "/Users/KimiZ/GRBs2/analysis/LAT/%s/PYXSPEC/%s/%s/"%(burst, detector, modname)

#fakedatafile = 

if not os.path.exists(os.path.join(data_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version))):
    

Does not exist


In [14]:
mod_dir

'/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/PYXSPEC/GBMwLAT/grbm+blackb/'

In [13]:
os.path.exists(os.path.join(mod_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version)))

False

In [12]:
if os.path.exists(os.path.join(data_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version))):  
    fakedatafile = os.path.join(data_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version))
elif os.path.exists(os.path.join(mod_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version))):
    fakedatafile = os.path.join(mod_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version))
else:
    print "Can't find file"
        

Can't find file


In [5]:
os.path.exists(os.path.join(data_dir, "fakeit_fitresults_%s_%s.fit"%(modname, version)))

False

In [4]:
Fdata = pyfits.open(fakedatafile)

iter = 0
Bdata = pd.DataFrame()
for name in Fdata[1].data.columns.names:
    Bdata["%s"%name] = Fdata[1].data["%s"%name]
    if 'PARAM' in name:
        iter += 1
    else:
        pass
nPars = iter

Bdata = Bdata.astype('float64')

IOError: File does not exist: '/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/integrated/fakeit_fitresults_grbm+blackb_-01-.fit'

In [ ]:
def Plot_ParamDistributions(modelname, parameter, Xlimits=None, bins=50):
    XLabel = str(parameter)

    x = Bdata["%s"%XLabel]
    mu, sigma = np.mean(x), np.sqrt(np.var(x))
    
    fig = plt.figure(figsize = (11,7))
    ax = fig.add_subplot(111)

    n, bins, patches = ax.hist(x, bins, normed=1, facecolor='green', alpha=0.75)
    bincenters = 0.5*(bins[1:]+bins[:-1])

    # add a 'best fit' line for the normal PDF
    y = mlab.normpdf(bincenters, mu, sigma)


    x1siglo = mu-sigma
    x1sighi = mu+sigma
    x2siglo = mu-2*sigma
    x2sighi = mu+2*sigma
    x3siglo = mu-3*sigma
    x3sighi = mu+3*sigma

    # ax.get_xlim()
    ylow, yhi = ax.get_ylim()
    ylabelpos = yhi - (yhi*.01)
    yhigh = yhi + (yhi*.1)
    #ylow, yhigh = ax.yaxis.get_data_interval()
    ax.vlines(mu, ylow, yhigh, linestyle='--', color="black") # mu + sigma
    ax.vlines(mu+sigma, ylow, yhigh, linestyle='--', color="black") # mu + sigma
    ax.vlines(mu-sigma, ylow, yhigh, linestyle='--', color="black") # mu - sigma
    ax.vlines(mu+2*sigma, ylow, yhigh, linestyle='--', color="black") # mu + 2*sigma
    ax.vlines(mu-2*sigma, ylow, yhigh, linestyle='--', color="black") # mu - 2*sigma
    ax.vlines(mu+3*sigma, ylow, yhigh, linestyle='--', color="black") # mu + 3*sigma
    ax.vlines(mu-3*sigma, ylow, yhigh, linestyle='--', color="black") # mu - 3*sigma
    #ax.text(x,y, "text")
    ax.text(mu, ylabelpos, r"$\mu$", fontsize = 12, horizontalalignment='center')
    ax.text(x1sighi, ylabelpos, r"$1\sigma$", fontsize = 12, horizontalalignment='center')
    ax.text(x1siglo, ylabelpos, r"$1\sigma$", fontsize = 12, horizontalalignment='center')
    ax.text(x2sighi, ylabelpos, r"$2\sigma$", fontsize = 12, horizontalalignment='center')
    ax.text(x2siglo, ylabelpos, r"$2\sigma$", fontsize = 12, horizontalalignment='center')
    ax.text(x3sighi, ylabelpos, r"$3\sigma$", fontsize = 12, horizontalalignment='center')
    ax.text(x3siglo, ylabelpos, r"$3\sigma$", fontsize = 12, horizontalalignment='center')
    ax.set_ylim(ylow, yhigh)
    
    ax.set_xlim(mu-5*sigma, mu+5*sigma)
    
    # if Xlimits is not None:
    #     ax.set_xlim(Xlimits)
    
    ax.plot(bincenters, y, 'r--', linewidth=1)
    ax.set_xlabel('%s'%XLabel)
    ax.set_title('%s'%modelname)


In [ ]:
for param in Bdata.columns:
    Plot_ParamDistributions('sbpl+bbody', param)

In [ ]:
mu         = np.mean(np.log(data))  # mean of the data
sigma     = np.std(np.log(data) )  # standard deviation of the data
mu, sigma

In [ ]:
XLabel = "PARAM3"

plt.figure(figsize=(13,8))

data = Bdata[XLabel]
if data.mean() < 0:
    data = abs(data)


count, bins, ignored = plt.hist(data, bins=100, normed=True, align='mid', alpha=0.2)

mu_norm         = np.mean(data)  # mean of the data
sigma_norm      = np.std(data)   # standard deviation of the data

x = np.linspace(min(bins), max(bins), 1000)
y = norm.pdf(x, mu_norm, sigma_norm)
plt.plot(x,y, 'b-')


mu_lognorm      = np.mean(np.log(data))  # the mean of the logged data
sigma_lognorm   = np.std(np.log(data))   # the standard deviation of the logged data

x = np.linspace(min(bins), max(bins), 1000)
y = Lognorm.pdf(x, mu_lognorm, sigma_lognorm)
plt.plot(x, y, 'r--')




norm_limits = norm.ppf([0.15865, 0.84135, 0.0227499999, 0.97725, 0.00135, 0.99865], mu_norm, sigma_norm)
plt.vlines(norm_limits, y.min(), y.max(), color='blue', linestyle='-', label="Normal")

lognorm_limits = Lognorm.quantile([0.15865, 0.84135, 0.0227499999, 0.97725, 0.00135, 0.99865], mu_lognorm, sigma_lognorm)
plt.vlines(lognorm_limits, y.min(), y.max(), color='red', linestyle='--', label="Lognormal")

#plt.xlim(200,800)

plt.legend(loc='best')
print ""
print "norm.ppf(prob, mu_norm, sigma_norm)"
print "                   (Lower Tail, Upper Tail)"
print "1-sigma_norm:  %.9f, %.9f"%(norm.ppf([0.15865], mu_norm, sigma_norm), norm.ppf([0.84135], mu_norm, sigma_norm))
print "2-sigma_norm:  %.9f, %.9f"%(norm.ppf([0.0227499999], mu_norm, sigma_norm), norm.ppf([0.97725], mu_norm, sigma_norm))
print "3-sigma_norm:  %.9f, %.9f"%(norm.ppf([0.00135], mu_norm, sigma_norm), norm.ppf([0.99865], mu_norm, sigma_norm))
print ""

print ""
print "Lognorm.quantile(prob, mu_lognorm, sigma_lognorm)"
print "                   (Lower Tail, Upper Tail)"
print "1-sigma_lognorm:  %.9f, %.9f"%(Lognorm.quantile([0.15865], mu_lognorm, sigma_lognorm), Lognorm.quantile([0.84135], mu_lognorm, sigma_lognorm))
print "2-sigma_lognorm:  %.9f, %.9f"%(Lognorm.quantile([0.0227499999], mu_lognorm, sigma_lognorm), Lognorm.quantile([0.97725], mu_lognorm, sigma_lognorm))
print "3-sigma_lognorm:  %.9f, %.9f"%(Lognorm.quantile([0.00135], mu_lognorm, sigma_lognorm), Lognorm.quantile([0.99865], mu_lognorm, sigma_lognorm))
print ""


Diff1_l  = np.diff([Lognorm.quantile(0.15865, mu_lognorm, sigma_lognorm), norm.ppf(0.15865, mu_norm, sigma_norm)])

Diff1_u  = np.diff([Lognorm.quantile(0.84135, mu_lognorm, sigma_lognorm), norm.ppf(0.84135, mu_norm, sigma_norm)])

Diff2_l  = np.diff([Lognorm.quantile(0.0227499999, mu_lognorm, sigma_lognorm), norm.ppf(0.0227499999, mu_norm, sigma_norm)])

Diff2_u  = np.diff([Lognorm.quantile(0.97725, mu_lognorm, sigma_lognorm), norm.ppf(0.97725, mu_norm, sigma_norm)])

Diff3_l  = np.diff([Lognorm.quantile(0.00135, mu_lognorm, sigma_lognorm), norm.ppf(0.00135, mu_norm, sigma_norm)])

Diff3_u  = np.diff([Lognorm.quantile(0.99865, mu_lognorm, sigma_lognorm), norm.ppf(0.99865, mu_norm, sigma_norm)])

print ""
print "Difference Between Normal Quantiles and Lognormal Quantiles"
print "                   (Lower Tail, Upper Tail)"
print "1-sigma_norm DIFF:  %.9f, %.9f"%(Diff1_l, Diff1_u)
print "2-sigma_norm DIFF:  %.9f, %.9f"%(Diff2_l, Diff2_u)
print "3-sigma_norm DIFF:  %.9f, %.9f"%(Diff3_l, Diff3_u)
print ""

print "\n"
print "From Distributions: "
print "Depending on which distribution is the best fit to the data, "
print "here are their statistical properties:"
print ""
print "MEAN: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.mean(mu_norm, sigma_norm), Lognorm.mean(mu_lognorm, sigma_lognorm))
print ""
print "MEDIAN: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.median(mu_norm, sigma_norm), Lognorm.median(mu_lognorm, sigma_lognorm))
print ""
print "MODE: (Normal, Lognormal)" # mode is the mean for the normal distribution
print "%.9f, %.9f"%(norm.mean(mu_norm, sigma_norm) , Lognorm.mode(mu_lognorm, sigma_lognorm))
print ""
print "STD DEV: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.std(mu_norm, sigma_norm), Lognorm.std(mu_lognorm, sigma_lognorm))
print ""
print "VARIANCE: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.var(mu_norm, sigma_norm), Lognorm.var(mu_lognorm, sigma_lognorm))
print ""

In [ ]:
np.std(Bdata["PARAM3"]), Bdata["PARAM3"].std()

In [ ]:
norm.std(mu_norm, sigma_norm)

In [ ]:
print "\n"
print "From Distributions: "
print "Depending on which distribution is the best fit to the data, "
print "here are their statistical properties:"
print ""
print "MEAN: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.mean(mu_norm, sigma_norm), Lognorm.mean(mu_lognorm, sigma_lognorm))
print ""
print "MEDIAN: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.median(mu_norm, sigma_norm), Lognorm.median(mu_lognorm, sigma_lognorm))
print ""
print "MODE: (Normal, Lognormal)" # mode is the mean for the normal distribution
print "%.9f, %.9f"%(norm.mean(mu_norm, sigma_norm) , Lognorm.mode(mu_lognorm, sigma_lognorm))
print ""
print "STD DEV: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.std(mu_norm, sigma_norm), Lognorm.std(mu_lognorm, sigma_lognorm))
print ""
print "VARIANCE: (Normal, Lognormal)"
print "%.9f, %.9f"%(norm.var(mu_norm, sigma_norm), Lognorm.var(mu_lognorm, sigma_lognorm))
print ""

In [ ]:
STAT_VAR = np.mean(Bdata["CSTAT"])
STAT_STD = np.std(Bdata["CSTAT"])
STAT_STD2 = Bdata["CSTAT"].std()

In [ ]:
STAT_VAR, STAT_STD, STAT_STD2

In [ ]:
Lognorm.mean(mu_lognorm, sigma_lognorm)

In [ ]:
norm.mean(mu_norm, sigma_norm)

In [ ]:
Diff1_l  = np.diff([Lognorm.quantile([0.15865], mu_lognorm, sigma_lognorm), norm.ppf(0.15865, mu_norm, sigma_norm)])
Diff1_l[0]

In [ ]:
mu_lognorm      = np.mean(np.log(abs(data)))  # the mean of the logged data
sigma_lognorm   = np.std(np.log(abs(data)))   # the standard deviation of the logged data

In [ ]:
mu_lognorm, sigma_lognorm

In [ ]:
mu_lognorm, sigma_lognorm

In [ ]:
Lognorm.quantile(.25, mu_lognorm, sigma_lognorm), norm.ppf(.25, mu_norm, sigma_norm)

In [ ]:
XLabel = "PARAM3"

x = Bdata["%s"%XLabel]

print ""
mu, sigma = np.mean(x), np.sqrt(np.var(x))
print "mu, sigma = np.mean(x), np.sqrt(np.var(x))"
print "mu: %f   sigma: %f " %(mu, sigma)
print ""

print ""
mu, sigma = x.mean(), x.std()
print "mu, sigma = x.mean(), x.std()"
print "mu: %f   sigma: %f " %(mu, sigma)
print ""


mu, sigma = np.mean(x), np.sqrt(np.var(x))
print ""
print "mu-sigma, mu+sigma"
print "1-sigma:  %.9f, %.9f"%(mu-sigma, mu+sigma)
print "2-sigma:  %.9f, %.9f"%(mu-2*sigma, mu+2*sigma)
print "3-sigma:  %.9f, %.9f"%(mu-3*sigma, mu+3*sigma)
print ""


print ""
print "ppf.norm(prob, mu, sigma)"
print "1-sigma:  %.9f, %.9f"%(norm.ppf([0.15865], mu, sigma), norm.ppf([0.84135], mu, sigma))
print "2-sigma:  %.9f, %.9f"%(norm.ppf([0.0227499999], mu, sigma), norm.ppf([0.97725], mu, sigma))
print "3-sigma:  %.9f, %.9f"%(norm.ppf([0.00135], mu, sigma), norm.ppf([0.99865], mu, sigma))
print ""


Diff1_l  = np.diff([mu-sigma, norm.ppf([0.15865], mu, sigma)])
Diff1_u  = np.diff([mu+sigma, norm.ppf([0.84135], mu, sigma)])

Diff2_l  = np.diff([mu-2*sigma, norm.ppf([0.0227499999], mu, sigma)])
Diff2_u  = np.diff([mu+2*sigma, norm.ppf([0.97725], mu, sigma)])

Diff3_l  = np.diff([mu-3*sigma, norm.ppf([0.00135], mu, sigma)])
Diff3_u  = np.diff([mu+3*sigma, norm.ppf([0.99865], mu, sigma)])


print ""
print "ppf.norm(prob, mu, sigma)"
print "1-sigma DIFF:  %.9f, %.9f"%(Diff1_l, Diff1_u)
print "2-sigma DIFF:  %.9f, %.9f"%(Diff2_l, Diff2_u)
print "3-sigma DIFF:  %.9f, %.9f"%(Diff3_l, Diff3_u)
print ""



In [ ]:
np.diff([mu-sigma, norm.ppf([0.15865], mu, sigma)])

In [ ]:
np.diff(mu-sigma, norm.ppf([0.15865], mu, sigma)[0])

In [ ]:
xA = Bdata["PARAM3"]
muA = Bdata["PARAM3"].mean()
Plot_ParamDistributions(modname, 'PARAM3', bins=100)
plt.plot(xA, poisson.pmf(xA, muA), 'go', ms=8, label='poisson pmf')
plt.vlines(524.111, 0, 0.005, colors='blue')
plt.vlines(443.6638, 0, 0.005, colors='blue')
plt.vlines(620.48059, 0, 0.005, colors='blue')
plt.vlines(375.74341, 0, 0.005, colors='blue')
plt.vlines(735.930723, 0, 0.005, colors='blue')
plt.xlim(200,1000)

In [ ]:
lognorm.fit(Bdata["PARAM3"]) # 0.20080073031167195, 29.38409602369115, 497.47526983981265

In [ ]:
def lognormpdf(x, mu, sigma):
    return (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2)) / (x * sigma * np.sqrt(2 * np.pi)))


plt.figure(figsize=(11,7))
b = Bdata["PARAM3"]
count, bins, ignored = plt.hist(b, 100, normed=True, align='mid')

sigma = np.std(np.log(b))
mu = np.mean(np.log(b))

print "mu: %f  sigma: %f"%(mu, sigma)

x = np.linspace(min(bins), max(bins), 10000)
# pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2)) / (x * sigma * np.sqrt(2 * np.pi)))

plt.plot(x, lognormpdf(x, mu, sigma), color='r', linewidth=2)

plt.vlines(524.111, 0, 0.005, colors='black')
plt.xlim(200,1000)
plt.show()

# lognorm.pdf doesn't have a mu parameter and therefore doesn't work.

lognorm.pdf(x, s) 

= 1 / (s\*x\*sqrt(2\*pi)) \* exp(-1/2\*(log(x)/s)^2)

pdf   
= (np.exp(-(np.log(x) - mu)^2 / (2 \* sigma^2)) / (x \* sigma \* np.sqrt(2 \* np.pi)))

In [ ]:
data = Bdata["PARAM3"]

sigma  = np.std(np.log(data));    # sigma is shape
mu     = np.mean(np.log(data));   # mu is loc
scale  = np.exp(mu);              # scale
print "sigma: %f   mu: %f   scale: %f"%(sigma, mu, scale)

In [ ]:
# lognorm.fit()  returns: shape(sigma), loc(mu), scale
sigma, mu, scale = lognorm.fit(data)
print "sigma: %f   mu: %f   scale: %f"%(sigma, mu, scale)

## Comparing the plots of the lognorm.fit(data) to using std, mean, and exp(mean) 
## to determine shape, loc, and scale.

In [ ]:
data = Bdata["PARAM3"]
plt.figure(figsize=(11,7))
count, bins, ignored = plt.hist(b, 100, normed=True, align='mid')

sigma, mu, scale = lognorm.fit(data)
x      = data
y_eqn  = (x - mu)
y      = lognorm.pdf(x, sigma, mu, scale)

plt.plot(x, y, 'g.', ms=6, label='lognorm.pdf using lognorm.fit(data)')


sigma   = np.std(np.log(data))
mu      = np.mean(np.log(data))
scale   = np.exp(mu);  

x      = data
y_eqn  = (x - mu)
y      = lognorm.pdf(x, sigma, mu, scale)

plt.plot(x, y, 'r.', ms=4, label='lognorm.pdf using std, mean, exp(mean) of data')

plt.xlim(200,1000)
plt.legend(loc='best')
plt.show()

In [ ]:
data = Bdata["PARAM3"]
plt.figure(figsize=(11,7))
count, bins, ignored = plt.hist(b, 100, normed=True, align='mid')

sigma   = np.std(np.log(data))
mu      = np.mean(np.log(data))
scale   = np.exp(mu);  
print "mu: %f  sigma: %f  scale: %f"%(mu, sigma, scale)

x      = data
y_eqn  = (x - mu)/scale
y      = lognorm.pdf(y_eqn, sigma)/scale
plt.plot(x, y, 'ro', ms=10)

x      = data
y_eqn  = (x - mu)
y      = lognorm.pdf(x, sigma, mu, scale)

plt.plot(x, y, 'g.', ms=4)

vals = lognorm.ppf([0.025, 0.5, 0.975], s=sigma, loc=mu, scale=scale)
print "P(0.025)= %f   P(.5)= %f    P(.975)= %f"%(vals[0], vals[1], vals[2])

plt.vlines(vals, y.min(), y.max(), colors='black', lw=4)
plt.xlim(200,1000)
plt.show()

In [ ]:
def lognorm_mean():
    return 

In [ ]:
sigma, mu, scale

In [ ]:
# no mode in numpy
np.mean(data), np.median(data), np.var(data)

In [ ]:
data.mean(), data.median(), data.mode(), data.var()

In [ ]:
# mean, median, mode, variance
np.exp(mu+(sigma**2)/2.), np.exp(mu), np.exp(mu-(sigma**2)), (np.exp(sigma**2)-1)*np.exp(2*mu+sigma**2)

In [ ]:
lognorm.mean(sigma, mu, scale), lognorm.median(sigma, mu, scale), lognorm.var(sigma, mu, scale)

In [ ]:
lognorm.mean(sigma, mu, scale), lognorm.median(sigma, mu, scale), lognorm.var(sigma, mu, scale)

In [ ]:
# mean
plt.vlines(524.111, 0, 0.005, colors='blue')

# 1 sigma
plt.vlines(443.6638, 0, 0.005, colors='blue')
plt.vlines(620.48059, 0, 0.005, colors='blue')

# 2 sigma
plt.vlines(375.74341, 0, 0.005, colors='blue')
plt.vlines(735.930723, 0, 0.005, colors='blue')

In [ ]:
vals = lognorm.ppf([0.025, 0.5, 0.975], s=sigma, loc=mu, scale=scale)
vals

In [ ]:
lognorm.ppf([0.34, 0.5, 0.65], s=sigma, loc=mu, scale=scale)

In [ ]:
# Generate a thousand samples: each is the product of 100 random
# values, drawn from a normal distribution.
# b = []
# for i in range(1000):
#     a = 10. + np.random.random(100)
#     b.append(np.product(a))

#b = np.array(b) / np.min(b) # scale values to be positive
plt.figure(figsize=(11,7))
b = Bdata["PARAM3"]
count, bins, ignored = plt.hist(b, 100, normed=True, align='mid')
sigma = np.std(np.log(b))
mu = np.mean(np.log(b))
print "mu: %f  sigma: %f"%(mu, sigma)

x = np.linspace(min(bins), max(bins), 10000)
pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2)) / (x * sigma * np.sqrt(2 * np.pi)))

plt.plot(x, pdf, color='r', linewidth=2)

plt.vlines(524.111, 0, 0.005, colors='blue')
plt.xlim(200,1000)
plt.show()

In [ ]:
lognorm.ppf([0.025], pdf)

In [ ]:
vals = lognorm.ppf([0.001, 0.5, 0.999], b.mean())

In [ ]:
vals

In [ ]:
xA = Bdata["PARAM3"]
muA = Bdata["PARAM3"].mean()
Plot_ParamDistributions(modname, 'PARAM3', bins=100)
plt.plot(xA, poisson.pmf(xA, muA), 'go', ms=8, label='poisson pmf')
plt.vlines(524.111, 0, 0.005, colors='blue')
plt.vlines(443.6638, 0, 0.005, colors='blue')
plt.vlines(620.48059, 0, 0.005, colors='blue')
plt.vlines(375.74341, 0, 0.005, colors='blue')
plt.vlines(735.930723, 0, 0.005, colors='blue')
plt.xlim(200,1000)

In [ ]:
# Generate a thousand samples: each is the product of 100 random
# values, drawn from a normal distribution.
b = []
for i in range(1000):
    a = 10. + np.random.random(100)
    b.append(np.product(a))

b = np.array(b) / np.min(b) # scale values to be positive
count, bins, ignored = plt.hist(b, 100, normed=True, align='mid')
sigma = np.std(np.log(b))
mu = np.mean(np.log(b))

x = np.linspace(min(bins), max(bins), 10000)
pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))
       / (x * sigma * np.sqrt(2 * np.pi)))

plt.plot(x, pdf, color='r', linewidth=2)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)
mu = Bdata["PARAM3"].mean() 

mean, var, skew, kurt = poisson.stats(mu, moments='mvsk') # mean is same as mu

x = np.arange(poisson.ppf(0.99, mu))#, poisson.ppf(0.99, mu))
ax.plot(x, poisson.pmf(x, mu), 'b-', label='poisson pmf')
ax.set_xlim([450,600])
#ax.vlines(x, 0, poisson.pmf(x, mu), colors='b', lw=5, alpha=0.5)

rv = poisson(mu)
ax.vlines(x, 0, rv.pmf(x), colors='k', linestyles='-', lw=1, label='frozen pmf')
ax.legend(loc='best', frameon=False)
plt.show()

In [ ]:
var

In [ ]:
#fig, ax = plt.subplots(1, 1)
mu = Bdata["PARAM3"].mean() 

mean, var, skew, kurt = poisson.stats(mu, moments='mvsk') # mean is same as mu

x = np.arange(poisson.ppf(0.99, mu))#, poisson.ppf(0.99, mu))
#x = np.arange(poisson.ppf(0.99, mu))#, poisson.ppf(0.99, mu))
Plot_ParamDistributions(modname, 'PARAM3', bins=100)
plt.plot(x, poisson.pmf(x, mu), 'b-', label='poisson pmf')
plt.xlim([200,1000])
#Plot_ParamDistributions(modname, 'PARAM3', bins=100)

In [ ]:
xA = Bdata["PARAM3"]
muA = Bdata["PARAM3"].mean()
Plot_ParamDistributions(modname, 'PARAM3', bins=100)
plt.plot(xA, poisson.pmf(xA, muA), 'go', ms=8, label='poisson pmf')
plt.vlines(524.111, 0, 0.005, colors='blue')
plt.vlines(443.6638, 0, 0.005, colors='blue')
plt.vlines(620.48059, 0, 0.005, colors='blue')
plt.vlines(375.74341, 0, 0.005, colors='blue')
plt.vlines(735.930723, 0, 0.005, colors='blue')

In [ ]:
for name in Bdata.columns:
    if 'PARAM' in name:
        Plot_ParamDistributions(modname, name)
        #plt.savefig(pp, format='pdf', dpi=150, bbox_inches = 'tight')
    elif ('AIC' in name) or ('BIC' in name) or ('LOGLIKE' in name) or ('CSTAT' in name):
        Plot_ParamDistributions(modname, name)
        #plt.savefig(pp, format='pdf', dpi=150, bbox_inches = 'tight')
    else:
        pass

In [ ]:
#pp.close()

In [ ]:
target = open(paramoutfile2, 'w')

In [ ]:
for name in Bdata.columns:
    if 'PARAM' in name:
        x = Bdata["%s"%name]
        muX, sigmaX = np.mean(x), np.sqrt(np.var(x))
        line1 =  "%s"%name
        line2 = "Value:    %.13f"%muX
        line3 = "1-sigma:  %.13f, %.13f"%(muX - sigmaX, muX + sigmaX)
        line4 = "2-sigma:  %.13f, %.13f"%(muX - 2*sigmaX, muX + 2*sigmaX)
        line5 = "3-sigma:  %.13f, %.13f"%(muX - 3*sigmaX, muX + 3*sigmaX)
        line6 = "min max:  %.13f, %.13f"%(x.min(), x.max())
        #print ""
        target.write(line1)
        target.write("\n")
        target.write(line2)
        target.write("\n")
        target.write(line3)
        target.write("\n")
        target.write(line4)
        target.write("\n")
        target.write(line5)
        target.write("\n")
        target.write(line6)
        target.write("\n")
        target.write("\n")
    elif ('AIC' in name) or ('BIC' in name) or ('LOGLIKE' in name) or ('CSTAT' in name):
        x = Bdata["%s"%name]
        muX, sigmaX = np.mean(x), np.sqrt(np.var(x))
        line1 =  "%s"%name
        line2 = "Value:    %.13f"%muX
        line3 = "1-sigma:  %.13f, %.13f"%(muX - sigmaX, muX + sigmaX)
        line4 = "2-sigma:  %.13f, %.13f"%(muX - 2*sigmaX, muX + 2*sigmaX)
        line5 = "3-sigma:  %.13f, %.13f"%(muX - 3*sigmaX, muX + 3*sigmaX)
        line6 = "min max:  %.13f, %.13f"%(x.min(), x.max())
        #print ""
        target.write(line1)
        target.write("\n")
        target.write(line2)
        target.write("\n")
        target.write(line3)
        target.write("\n")
        target.write(line4)
        target.write("\n")
        target.write(line5)
        target.write("\n")
        target.write(line6)
        target.write("\n")
        target.write("\n")
    else:
        pass

In [ ]:
target.close()

In [ ]:
param_dict = OrderedDict()
for name in Bdata.columns:
    if 'PARAM' in name:
        x = Bdata["%s"%name]
        muX, sigmaX = np.mean(x), np.sqrt(np.var(x))
        # print "%s"%name
        # print "Value:    %.13f"%muX
        # print "1-sigma:  %.13f, %.13f"%(muX - sigmaX, muX + sigmaX)
        # print "2-sigma:  %.10f, %.10f"%(muX - 2*sigmaX, muX + 2*sigmaX)
        # print "3-sigma:  %.10f, %.10f"%(muX - 3*sigmaX, muX + 3*sigmaX)
        # print ""
        param_dict["%s"%name] = [muX, muX - sigmaX, muX + sigmaX, muX - 2*sigmaX, muX + 2*sigmaX, muX - 3*sigmaX, muX + 3*sigmaX, x.min(), x.max()]
    elif ('AIC' in name) or ('BIC' in name) or ('LOGLIKE' in name) or ('CSTAT' in name):
        x = Bdata["%s"%name]
        muX, sigmaX = np.mean(x), np.sqrt(np.var(x))
        param_dict["%s"%name] = [muX, muX - sigmaX, muX + sigmaX, muX - 2*sigmaX, muX + 2*sigmaX, muX - 3*sigmaX, muX + 3*sigmaX, x.min(), x.max()]
    else:
        pass
        

In [ ]:
outDataFrame = pd.DataFrame(param_dict)

In [ ]:
outDataFrame.to_csv(path_or_buf = paramoutfile, sep = ',', header=True, index=False)

In [ ]:
os.system('cp %s %s'%(outfile, outfile2))